## [Yolov5] Custom Dataset으로 YOLO 학습
- 목표 : 잎 탐지 전용 Yolo모델
- 활용 데이터 셋 : hub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=153
- GCP를 활용하여 대규모 데이터셋을 학습
  - GCP Vertex AI
    - GCP에서 제공하는 통합 AI 서비스
    - Voertex AI - Colab Enterprise 사용
      - 런타임 템플릿
        - 별도의 인증없이 사용가능한 g2-standard-4 가속기 사용
        - GPU 유형 : NVIDIA_L4 * 1개
        - 스토리지 용량 : 2000GB
  - 데이터 요약
    - 모든 데이터는 총 45601개의 이미지 및 라벨링 데이터
      - 식물의 잎의 정보를 가지고있는 데이터는 총 28907개
    - 토마토, 고추, 상추 데이터 존재
    - 학습, 검증, 테스트 데이터 분류 기준
      - 학습 : 전체이미지의 95% (27461개)
      - 검증 : 전체이미지의 3% (867개)
      - 테스트 : 전체이미지의 2% (579개)
  - 에포크 : 40
  



### 기존에 구축하였던 Yolo 커스텀 데이터셋 학습 폴더

- Yolo train 폴더 구조
  - CustomDataset <br>
  ㄴ train/<br>
  &nbsp;&nbsp;&nbsp; 1.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 1.txt(어노테이션 파일)<br>
  ㄴ valid/<br>
  &nbsp;&nbsp;&nbsp; 2.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 2.txt(어노테이션 파일)<br>
  ㄴ test/<br>
  &nbsp;&nbsp;&nbsp; 3.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 3.txt(어노테이션 파일)<br>
  config.yaml<br>

#### 폴더안의 모든 이미지 파일 및 어노테이션 파일 경로 읽기
- 이 함수는 지정된 루트 디렉토리에서 서브셋으로 지정된 하위 디렉토리들에 대해 지정된 확장자를 가진 파일들의 리스트를 가져옵니다. 이를 통해 커스텀 데이터셋의 이미지 파일 및 어노테이션 파일의 경로를 쉽게 얻을 수 있습니다.
- 매개변수
  - root_dir : 파일을 검색할 루트 디렉토리 경로입니다.
  - subsets : 검색할 하위 디렉토리 이름을 포함하는 리스트입니다. 주로 "train", "valid", "test"와 같이 학습, 검증, 테스트 등의 하위 데이터셋을 지정합니다.
  - extensions : 검색할 파일의 확장자를 포함하는 리스트입니다. 예를 들어, "json", "txt", "jpg", "png" 등이 될 수 있습니다.

In [ ]:
import glob
import os

def get_subsets_file_list(root_dir, subsets, extensions):
    file_lists = {subset: [] for subset in subsets}

    for subset in subsets:
        for ext_label in extensions:
            file_lists[subset].extend(glob.glob(os.path.join(root_dir, subset, f"*.{ext_label}")))

    return file_lists

#### 파일 리스트를 텍스트 파일로 변환하기
- 이 함수는 파일 리스트를 텍스트 파일로 저장하는 기능을 합니다. 각 파일 리스트는 하나의 텍스트 파일에 한 줄씩 저장됩니다.

- 매개변수
  - file_list: 저장할 파일 경로 리스트입니다.
  - output_path: 파일 리스트를 저장할 텍스트 파일의 경로입니다.

In [ ]:
def write_file_list_to_txt(file_list, output_path):
    if os.path.exists(output_path):
        os.remove(output_path)
    with open(output_path, 'w') as f:
        f.write('\n'.join(file_list) + '\n')

### 실행

#### 환경 변수 설정
- root_dir: 데이터셋이 위치한 루트 디렉토리의 경로입니다. 이 디렉토리 안에는 훈련, 검증, 테스트 데이터셋이 각각의 하위 디렉토리로 구성되어 있습니다.
- extensions: 데이터셋에서 사용되는 파일의 확장자를 지정한 리스트입니다. 여기서는 주로 이미지 파일과 어노테이션 파일을 나타내는 확장자들이 포함됩니다.
- subsets: 데이터셋을 훈련, 검증, 테스트 등의 서브셋으로 분할할 때 사용할 서브셋의 이름을 포함하는 리스트입니다.
- yaml_path: YAML 파일을 저장할 경로를 지정합니다. 이 파일은 설정된 데이터셋의 구성을 저장하게 됩니다.
- save_path: 모델을 학습하고 나서 생성된 가중치 파일을 저장할 경로

In [ ]:
root_dir = "/content/YOLO_CustomDataSet_V2/"
extensions = ["json", "txt", "jpg", "JPG", "png", "PNG"]
subsets = ["train", "valid", "test"]
yaml_path = os.path.join(root_dir, "config.yaml")
save_path = "/content/models/leaf_detection.pt"

#### (번외) YOLO 형식의 어노테이션 파일(.txt)의 일부 내용을 바꾸기
  - 폴더 구축과정에서 실수로 class code를 잘 못 설정하였을때 각 subsets 폴더에 직접 어노테이션 파일을 업데이트

In [ ]:
import os

def map_plant_codes(folder_path, plant_code_mapping):
    # 폴더 내의 모든 .txt 파일을 찾습니다.
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            lines = []

            # 각 .txt 파일을 읽어서 매핑을 적용합니다.
            with open(file_path, "r") as file:
                for line in file:
                    class_index, *values = line.strip().split(" ")
                    mapped_class_index = plant_code_mapping.get(class_index, class_index)
                    new_line = f"{mapped_class_index} {' '.join(values)}"
                    lines.append(new_line)

            # 수정된 내용을 파일에 씁니다.
            with open(file_path, "w") as file:
                file.write("\n".join(lines))

In [ ]:
# 0: 고추, 1: 토마토, 11: 상추
for subfolder in subsets:
  folder_path = "/content/YOLO_CustomDataSet_V2/" + subfolder
  plant_code_mapping = {"02": "0", "05": "1", "11": "2"}

  # 함수를 호출하여 매핑 작업을 수행합니다.
  map_plant_codes(folder_path, plant_code_mapping)

#### 학습 폴더 구축

In [ ]:
# 하위폴더에 있는 파일의 경로를 읽는다.
file_lists = get_subsets_file_list(root_dir, subsets, extensions)

# 각 서브셋 폴더에 해당하는 파일 경로를 저장한 텍스트 파일 생성
for subfolder in subsets:
  write_file_list_to_txt(file_lists[subfolder], os.path.join(root_dir, f"{subfolder}.txt"))

##### config.yaml 파일 생성
- YAML 형식으로 데이터를 저장하는 과정입니다.

- data 딕셔너리에는 다음 내용이 포함되어 있습니다:

  - "train": 훈련 데이터셋 파일 경로 (train.txt)
  - "val": 검증 데이터셋 파일 경로 (valid.txt)
  - "test": 테스트 데이터셋 파일 경로 (train.txt)
  - "names": 클래스 이름과 해당 인덱스를 매핑한 딕셔너리
- yaml.dump() 함수를 사용하여 이 데이터를 YAML 형식으로 파일에 저장합니다. 파일 경로는 yaml_path 변수에 지정되어 있습니다.

- 검증
  - 저장된 YAML 파일을 확인하기 위해, yaml.safe_load() 함수를 사용하여 파일을 읽고 그 내용을 출력합니다.

In [ ]:
import yaml
class_names = {0: '고추', 1: '토마토', 2: '상추'}
data = {
    "train": os.path.join(root_dir, "train.txt"),
    "val": os.path.join(root_dir, "valid.txt"),
    "test": os.path.join(root_dir, "train.txt"),
    "names": class_names
}
with open(yaml_path, 'w') as f :
    yaml.dump(data, f)

# 저장된 yaml파일 검증
with open(yaml_path, 'r') as f :
    lines = yaml.safe_load(f)
    print(lines)

{'names': {0: 'leaf'}, 'test': '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet/train.txt', 'train': '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet/train.txt', 'val': '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet/valid.txt'}


#### YOLO 모델 불러오기
- ultralytics는 딥러닝 및 컴퓨터 비전 작업을 위한 파이썬 패키지입니다. 주로 객체 검출(Object Detection), 객체 추적(Object Tracking), 세그멘테이션(Segmentation) 등의 작업을 위한 유틸리티 및 모델을 제공합니다. 특히, YOLO(You Only Look Once) 객체 검출 알고리즘을 구현한 모델 및 관련 도구들을 제공하여 객체 검출 작업을 쉽게 수행할 수 있습니다.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.1/752.1 kB 11.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from ultralytics import YOLO

# 미리 학습된 YOLOv8 모델 가중치와 구조를 불러옵니다.
model = YOLO('yolov8s.pt')

# YAML 파일에 있는 정보를 사용하여 모델을 fine-tuning 합니다. 학습을 20 에포크 동안 진행합니다.
model.train(data=yaml_path, epochs=20)

100%|██████████| 21.5M/21.5M [00:00<00:00, 202MB/s]


Ultralytics YOLOv8.2.4 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/YOLO_CustomDataSet/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None

100%|██████████| 755k/755k [00:00<00:00, 24.9MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 107MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      16/20      4.19G     0.3479     0.3945      0.941          1        640: 100%|██████████| 18/18 [00:26<00:00,  1.48s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.23it/s]

                   all          9          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      17/20      4.11G     0.3358     0.4018     0.8875         16        640:  33%|███▎      | 6/18 [00:08<00:16,  1.37s/it]

#### 학습된 가중치 파일을 저장

In [ ]:
# 모델 객체 생성
model = YOLO('/content/runs/detect/train/weights/best.pt')

# 학습된 가중치 파일을 특정 경로에 저장
model.save(save_path)